**Import Libraries**

In [6]:
import os
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import DebertaTokenizer, DebertaForSequenceClassification
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit

**Mound Drive**

In [14]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Load The Dataset**

In [2]:
dataset = pd.read_csv("dataset_after_preprocessing.csv")

**Encode the labels**

In [3]:
# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'Assignee' column to numeric labels
dataset['label'] = label_encoder.fit_transform(dataset['Assignee'])

In [4]:
dataset

,Summary_Stemmed,Assignee,label
0,"['scroll', 'scroll', 'mice', 'touchpad', 'etc'...",amit@chromium.org,151
1,"['proxi', 'caus', 'network', 'request', 'fail'...",jon@chromium.org,1326
2,"['web', 'inspector', 'button', 'dock', 'main',...",pfeldman@chromium.org,2039
3,"['habari', 'admin', 'interfac', 'render', 'cor...",jon@chromium.org,1326
4,"['maxim', 'second', 'larger', 'monitor', 'work...",pkasting@chromium.org,2061
...,...,...,...
197914,"['updat', 'gleanj', 'dashboard', 'ignor', 'gle...",brosa,350
197915,"['autocomplet', 'type', 'valid', 'valu', 'pass...",brosa,350
197916,"['intermitt', 'slow', 'see', 'ping', 'show', '...",brosa,350
197917,"['investig', 'string', 'metric', 'type', 'adeq...",pmcmanis,2070


**Split the dataset**

In [9]:
# Create the StratifiedShuffleSplit object
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
# Split the dataset into train and test sets
train_idx, test_idx = next(sss.split(dataset, dataset['label']))
train_df = dataset.iloc[train_idx].reset_index(drop=True)
test_df = dataset.iloc[test_idx].reset_index(drop=True)

In [10]:
# Initialize the tokenizer
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

In [11]:
# Dataset class
class TextDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['Summary_Stemmed']
        label = self.dataframe.iloc[idx]['label']
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [12]:
# Create datasets and dataloaders
train_dataset = TextDataset(train_df, tokenizer, max_len=128)
test_dataset = TextDataset(test_df, tokenizer, max_len=128)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [13]:
# Initialize the model
num_labels = len(label_encoder.classes_)
model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=num_labels)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

pytorch_model.bin:   0%|          | 0.00/559M [00:00<?, ?B/s]

Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [15]:
# Directory to save checkpoints
CHECKPOINT_DIR = '/content/drive/MyDrive/checkpoints'

if not os.path.exists(CHECKPOINT_DIR):
    os.makedirs(CHECKPOINT_DIR)

In [16]:
# Training function with checkpoint saving
def train_epoch(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples, epoch):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:
        input_ids = d['input_ids'].to(device)
        attention_mask = d['attention_mask'].to(device)
        labels = d['label'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        logits = outputs.logits

        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Save the model and optimizer state after each epoch
    checkpoint_path = os.path.join(CHECKPOINT_DIR, f'checkpoint_epoch_{epoch}.pt')
    torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': np.mean(losses),
    }, checkpoint_path)

    return correct_predictions.double() / n_examples, np.mean(losses)

In [17]:
# Evaluation function
def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d['input_ids'].to(device)
            attention_mask = d['attention_mask'].to(device)
            labels = d['label'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=labels
            )
            loss = outputs.loss
            logits = outputs.logits

            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

    return correct_predictions.double() / n_examples, np.mean(losses)

In [18]:
# Loss function
loss_fn = torch.nn.CrossEntropyLoss().to(device)

# Load checkpoint if available
def load_checkpoint(model, optimizer, checkpoint_path):
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        epoch = checkpoint['epoch']
        loss = checkpoint['loss']
        print(f"Checkpoint loaded. Resuming training from epoch {epoch} with loss {loss:.4f}.")
        return epoch
    else:
        print("No checkpoint found. Starting training from scratch.")
        return 0

In [ ]:
# Load the last checkpoint
last_checkpoint_path = os.path.join(CHECKPOINT_DIR, 'checkpoint_epoch_3.pt')
start_epoch = load_checkpoint(model, optimizer, last_checkpoint_path)

# Training loop
EPOCHS = 100

for epoch in range(start_epoch, EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(
        model,
        train_loader,
        loss_fn,
        optimizer,
        device,
        None,
        len(train_df),
        epoch + 1
    )

    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(
        model,
        test_loader,
        loss_fn,
        device,
        len(test_df)
    )

    print(f'Val   loss {val_loss} accuracy {val_acc}')

No checkpoint found. Starting training from scratch.
Epoch 1/100
----------
